# NB: Introducing Pandas II

## Set Up

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
iris = sns.load_dataset('iris')

In [ ]:
iris.head(2)

In [ ]:
import sys
sys.getsizeof(iris)

## Apply Lambda Functions with `.apply()`

Apply a transformation to each record. Uses a `lambda` function.

The `apply()` method should be used after you have established that you can't use a vectorized function.

In [ ]:
iris['sepal_len_sq'] = iris.sepal_length.apply(lambda x: x**2)
iris.head(5)

Transformation involving multiple columns. Uses `axis=1` to access columns.  
Compute average of `sepal_length`, `sepal_width`:

In [ ]:
iris['sepal_len_wid_avg'] = iris[['sepal_length','sepal_width']].apply(lambda x: (x.sepal_length+x.sepal_width)/2, axis=1)
iris.head()

**Vectorized Version**

In [ ]:
%time iris.sepal_length**2

Compare to `.apply()`

In [ ]:
%time iris.sepal_length.apply(lambda x: x**2)

## Aggregation

Involves one or more of:

- splitting the data into groups
- applying a function to each group
- combining results

### `.groupby()`

Compute mean of each column, grouped (separately) by species

In [ ]:
iris.groupby("species").mean()

### `pd.pivot_table()`

Apply a function `aggfunc` to selected values grouped by columns

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)

Compute mean sepal length for each species:

In [ ]:
pd.pivot_table(iris, values="sepal_length", columns=["species"], aggfunc = np.mean)

## Stacking and Unstacking

Similar to pivoting, but requires -- and takes advantage of -- indexes.

In [ ]:
iris_w_idx = iris.copy() 

## Give the original index a name
iris_w_idx.index.name = 'obs_id'

## Create a multi-index, using `species` as part of the key.
iris_w_idx = iris_w_idx.reset_index().set_index(['species','obs_id'])

In [ ]:
iris_w_idx

### `.unstack()`

[Details](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html)

In [ ]:
iris_wide = iris_w_idx.sepal_length.unstack(fill_value=0).T

In [ ]:
iris_wide

In [ ]:
iris_wide.mean()

### `.stack()`

[Details](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.stack.html)

In [ ]:
iris_wide.T.stack().to_frame('sepal_length')

## Combining DataFrames

### `pd.concat()`  

Concatenate pandas objects along an axis.

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

Create two dfs and vertically stack them

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4))
df2 = pd.DataFrame(np.random.randn(3, 4))

In [ ]:
df1

In [ ]:
df2

**Concat rows**

In [ ]:
df3 = pd.concat([df1, df2], axis=0)

In [ ]:
df3

**Concat columns**

This assumes that the indexes represent IDs of specific things or events.

In [ ]:
df4 = pd.concat([df1, df2], axis=1, keys=['foo', 'bar'])

In [ ]:
df4

In [ ]:
df4.foo

### `.merge()`

SQL-style joining of tables (DataFrames) -- although Pandas has a `.join()` method, too.

Important parameters include:

- `how` : type of merge {'left', 'right', 'outer', 'inner', 'cross'}, default ‘inner’
- `on`  : names to join on
        
[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

Create two tables, `left` and `right`. Then right join them on `key`.  
Right join means include all records from table on right.  
The `key` is used for matching up the records.

In [ ]:
##| tags: []
left = pd.DataFrame({"key": ["jamie", "bill"], "lval": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "rval": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

Notice the NaN inserted into the record with key=asher, since the left table didn't contain the key.

**Matching column names**  
In this next example, the value columns have the same name: *val*.  Notice what happens to the column names.

In [ ]:
##| tags: []
left = pd.DataFrame({"key": ["jamie", "bill"], "val": [15, 22]})
right = pd.DataFrame({"key": ["jamie", "bill", "asher"], "val": [4, 5, 8]})

joined = pd.merge(left, right, on="key", how="right")

print('---left')
print(left)
print('\n---right')
print(right)
print('\n---joined')
print(joined)

### `.join()`

An SQL-like joiner, but this one takes advantage of indexes.

Give our dataframes indexes and distinctive columns names.

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)

In [ ]:
left2 = left.set_index('key').rename(columns={'val':'val_1'})
right2 = right.set_index('key').rename(columns={'val':'val_2'})

In [ ]:
left2

In [ ]:
right2

In [ ]:
right2.join(left2) # Defaults to 'inner'

In [ ]:
right2.join(left2, how='left')

### Summary

* Use **join** if you have shared indexes
* Use **merge** if you do not have shared indexes
* Use **concat** to combine based on shared indexes or columns

## Reshape with `.reshape()`

Changes the object's shape

We illustrate creating pandas Series, extracting array of length 6, and reshaping to 3x2 array.

In [ ]:
## create a series 
ser = pd.Series([1, 1, 2, 3, 5, 8]) 

## extract values 
vals = ser.values 

print('orig data:', vals)
print('orig type:', type(vals))
print('orig shape:', vals.shape)

## reshaping series
reshaped_vals = vals.reshape((3, 2)) 

print('\n reshaped vals:')
print(reshaped_vals)
print('\n new type:', type(reshaped_vals))
print('new shape:', reshaped_vals.shape)

Including -1 as one of the dimensions tells numpy: infer this dimension from the data and the other dimensions.

Example: enforce 3 columns:

In [ ]:
vals.reshape(-1,3)

Enforce 3 rows:

In [ ]:
vals.reshape(3,-1)

**IMPORTANT NOTE**  

Notice the shape of original array: `(6,)`  
This is a vector with one dimension, and is different from two-dimensional `(6,1)` array

## Categoricals

Categorical data takes discrete values where computation on the values does not make sense.  
Zip code is a typical example.

To include categoricals in models, they must be converted to numeric.  

### `get_dummies()`
Dummy code categorical data

Important parameters: 

- `prefix`    : append prefix to column names (a good idea for later use)
- `drop_first`: remove first level, as only `k-1` variables needed to represent `k` levels

[Details](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

In [ ]:
cats = pd.DataFrame({'breed':['persian','persian','siamese','himalayan','burmese']})

In [ ]:
cats

In [ ]:
dummy_cats = pd.get_dummies(cats.breed, drop_first=True, prefix='breed')

In [ ]:
dummy_cats

Notice `burmese` was dropped (first level by alphabet) since it can be inferred.
